# Connecting to NAO

NAO is an autonomous, programmable humanoid robot developed by Aldebaran Robotics, a French robotics company. NAO is the world's leading and most widely used humanoid robot for education, healthcare, and research. NAO is 58cm tall, autonomous, and fully programmable robot that can walk, talk, listen to you, and even recognise your face. NAO has advanced platforms for the in-depth study such as human-machine interaction, cognitive computing and autonomous navigation, etc.

First step of working with the NAO robot involves connecting to it through the internet which can be done either via an ethernet cable or a WiFi connection. However, for convenience we will be connecting to NAO via the WiFi which requires both the NAO and our computer(from which we will be executing the commands) to be on the same network. 

### Power ON 

Plug in the power cable behind the NAO and press the Power button in the chest for 3 seconds and wait for a couple of minutes for the NAO to boot up.

<img src="./images/wiz_step3.png"/>

<img src="./images/power-on.png"/>

### Getting NAO's IP

After NAO's up it will announce 'OGNAK GNUK'. Now you can press the Power button on the chest once to make NAO announce its IP. We will be using this IP to connect to the NAO. 

NOTE: NOA's IP might everytime it boots up to connect to the network. Hence, its good practice to verify NAO's IP every time it boots up.

<img src="./images/note_ipaddress.png">

### Verifying NAO's IP

You can also verify the NAO's IP by entering the IP in your browser when you are connected to the same network as NAO and you will get a page with popup asking your authentication as below.


<img src="./images/verify-ip.png">

### Changing NAO's WiFi Connection

In case you want to change the network NAO is connected, you can login in this page with the authentication credentials that will be provided in the back of your NAO.

After successfull login, you can proceed to change the WiFi connection under Network Setup section.


<img src="./images/nao-wifi.png">

### Making NAO Speak
 
To run our first script on NAO, we can make NAO say 'Hello World!'

But first, make sure you have NAOqi python package installed on your system which will let us write python script to run and interact with NAO.

In the below code snippet, change `<NAO_IP>` to your NAO's IP and `<YOUR_NAME>` to your name, Duh!. 

For example, if your NAO's IP is 192.168.1.7 then change the second line of the code snippet so that it looks like below and run it. Your NAO should now say 'Hello world'.

```
tts = ALProxy("ALTextToSpeech", "192.168.1.7", 9559)
```

and if your name is 'Tom Cruise' then
```
tts.say("Hello, Tom Cruise!")
```


In [1]:
from naoqi import ALProxy

#tts = ALProxy("ALTextToSpeech", "192.168.1.7", 9559)
tts = ALProxy("ALTextToSpeech", "<NAO_IP>", 9559)
tts.say("Hello, <YOUR_NAME>!")

RuntimeError: 	ALBroker::createBroker
	Cannot connect to tcp://<NAO_IP>:9559

Feeling lucky? Try changing the text from "Hello, world!" to anything you like and make NOA announce it.

Congratulations, you have now successfully run your first python script on NAO.

### How it Works

The above script uses the say method of the `ALTextToSpeech` module. `ALTextToSpeech` is the module of NAoqi dedicated to speech. The say method makes the robot pronounce the string given in parameter.

`from naoqi import ALProxy` 

This line imports the module ALProxy.


`tts = ALProxy("ALTextToSpeech", "<NAO_IP>", 9559)`

This line creates an object called tts. This object will send calls to NAOqi.

    * tts is the name we gave to the object instance (could have been myspeechmodule or speakingmodule).
    * ALProxy() is a class of objects, allowing you to have acces to all the methods of a module.
    * ALTextToSpeech is the name of the module of NAOqi we want to use.
    * IP and Port (9559) of the robot are also specified (it was not the case with Choregraphe).


`tts.say("Hello, world!")`

This line uses the object tts to send an instruction to the NAOqi module.

    * tts is the object we use.
    * say() is the method.
    * “Hello, world!” is the parameter.



### Making NAO Move

To make NAO walk, you should use `ALMotionProxy::moveInit()` (to put the robot in a correct position), and then `ALMotionProxy::moveTo()`

Run the below snippet to make NAO walk 0.5 meters forward. Oh and clear a path!

In [ ]:
from naoqi import ALProxy

#motion = ALProxy("ALMotion", "192.168.1.7", 9559)
motion = ALProxy("ALMotion", "<NAO_IP>", 9559)
motion.moveInit()
motion.moveTo(0.5, 0, 0)

The function `ALMotionProxy::moveTo(x,y,theta)` makes the robot move to the given pose in the ground plane, relative to current position of the robot. This is a blocking call.

Parameters:	

    x – Distance along the X axis in meters.
    y – Distance along the Y axis in meters.
    theta – Rotation around the Z axis in radians [-3.1415 to 3.1415].


### Parallel Tasks

You can make your NAO do several things at a time. Now lets make NAO move and speak at the same time.



In [ ]:
from naoqi import ALProxy

#motion = ALProxy("ALMotion", "192.168.1.7", 9559)
#tts    = ALProxy("ALTextToSpeech", "192.168.1.7", 9559)
motion = ALProxy("ALMotion", "<NAO_IP>", 9559)
tts    = ALProxy("ALTextToSpeech", "<NAO_IP>", 9559)
motion.moveInit()
motion.post.moveTo(0.5, 0, 0)
tts.say("I'm walking")

If you need to wait until a given task is finished, you can use the wait method of ALProxy, using the ID of the tasked returned by the post usage like below.

In [ ]:
from naoqi import ALProxy

#motion = ALProxy("ALMotion", "192.168.1.7", 9559)
#tts    = ALProxy("ALTextToSpeech", "192.168.1.7", 9559)
motion = ALProxy("ALMotion", "<NAO_IP>", 9559)
tts    = ALProxy("ALTextToSpeech", "<NAO_IP>", 9559)
motion.moveInit()
id = motion.post.moveTo(0.5, 0, 0)
motion.wait(id, 0)
tts.say("I have arrived")

NOTE: Also, some tasks will enforce NAO's joints to be stiff and the robot will not move unless you set the stiffness of the joints to something that is not 0.

In [ ]:
from naoqi import ALProxy

#motion = ALProxy("ALMotion", "192.168.1.7", 9559)
motion = ALProxy("ALMotion", "<NAO_IP>", 9559)
motion.setStiffnesses("Body", 1.0)

### Reacting to Events

Here we want the robot say ‘I see you’ every time it detects a human face.To do this, we need to subscribe to the ‘FaceDetected’ event, raised by the ALFacedetection module. The `FaceDetected` event triggers every time the NAO sees a face. You can try covering your face, it will not announce.

Replace `<NAO_IP>` with your NAO's IP.
 
You can run the script below, which will run an infinite loop that will keep anouncing as long as it keeps detecting faces. You can stop it by pressing the Stop button in Jupyter Notebook.


In [ ]:
import sys
import time

from naoqi import ALProxy
from naoqi import ALBroker
from naoqi import ALModule

from optparse import OptionParser

#NAO_IP = "192.168.1.7"
NAO_IP = "<NAO_IP>"

# Global variable to store the HumanGreeter module instance
HumanGreeter = None
memory = None


class HumanGreeterModule(ALModule):
    """ A simple module able to react
    to facedetection events

    """
    def __init__(self, name):
        ALModule.__init__(self, name)
        # No need for IP and port here because
        # we have our Python broker connected to NAOqi broker

        # Create a proxy to ALTextToSpeech for later use
        self.tts = ALProxy("ALTextToSpeech")

        # Subscribe to the FaceDetected event:
        global memory
        memory = ALProxy("ALMemory")
        memory.subscribeToEvent("FaceDetected",
            "HumanGreeter",
            "onFaceDetected")

    def onFaceDetected(self, *_args):
        """ This will be called each time a face is
        detected.

        """
        # Unsubscribe to the event when talking,
        # to avoid repetitions
        memory.unsubscribeToEvent("FaceDetected",
            "HumanGreeter")

        self.tts.say("I see you!")

        # Subscribe again to the event
        memory.subscribeToEvent("FaceDetected",
            "HumanGreeter",
            "onFaceDetected")
        
    
if __name__ == "__main__":
    myBroker = ALBroker("myBroker",
       "0.0.0.0",   # listen to anyone
       0,           # find a free port and use it
       NAO_IP,         # parent broker IP
       9559)       # parent broker port


    # Warning: HumanGreeter must be a global variable
    # The name given to the constructor must be the name of the
    # variable
    global HumanGreeter
    HumanGreeter = HumanGreeterModule("HumanGreeter")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print
        print "Interrupted by user, shutting down"
        myBroker.shutdown()
        sys.exit(0)